In [133]:
import pandas as pd
import numpy as np

In [134]:
data = pd.read_csv('reviews_det_sentiment.csv')
# print(data.head(10))

In [135]:
listings = pd.read_csv('listings_det.csv')
print(listings.head(10))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (43,61,62,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


     id                        listing_url       scrape_id last_scraped  \
0  2515  https://www.airbnb.com/rooms/2515  20180303203649   2018-03-04   
1  2539  https://www.airbnb.com/rooms/2539  20180303203649   2018-03-04   
2  2595  https://www.airbnb.com/rooms/2595  20180303203649   2018-03-04   
3  3330  https://www.airbnb.com/rooms/3330  20180303203649   2018-03-04   
4  3647  https://www.airbnb.com/rooms/3647  20180303203649   2018-03-04   
5  3831  https://www.airbnb.com/rooms/3831  20180303203649   2018-03-04   
6  4989  https://www.airbnb.com/rooms/4989  20180303203649   2018-03-04   
7  5054  https://www.airbnb.com/rooms/5054  20180303203649   2018-03-06   
8  5121  https://www.airbnb.com/rooms/5121  20180303203649   2018-03-04   
9  5136  https://www.airbnb.com/rooms/5136  20180303203649   2018-03-05   

                                             name  \
0                Stay at Chez Chic budget room #1   
1              Clean & quiet apt home by the park   
2              

In [121]:
# print(listings.columns.values)
# print(listings['id'])
# true_scores = listings[['id','review_scores_rating']]
# print(true_scores.shape)
# true_scores = true_scores.dropna(axis=0, how='any')
# print(true_scores.shape)

In [136]:
# data['true_score'] = np.nan
# for index, row in data.iterrows():
# #     print(row['listing_id'])
#     score = true_scores.loc[true_scores['id'] == row['listing_id']]['review_scores_rating'].tolist()
#     if len(score) > 0:
# #         print(score)
#         data.set_value(index, 'true_score', score[0])
# #     print(score)
# print(data.head(10))

data = data.merge(listings, left_on='listing_id', right_on='id')
data = data.filter(['listing_id','reviewer_id', 'sentiment', 'review_scores_rating'], axis=1)
data.rename(columns={'review_scores_rating':'true_score'}, inplace=True)
data = data.dropna(axis=0, how='any')
data['true_score'] = data['true_score']/100
data['sentiment'] = (data['sentiment'] + 1) / 2
print(data.head(10))

   listing_id  reviewer_id  sentiment  true_score
0        2515         2603    0.96785        0.92
1        2515         8455    0.97685        0.92
2        2515         9759    0.98100        0.92
3        2515         9193    0.98130        0.92
4        2515         7048    0.96675        0.92
5        2515        12521    0.97505        0.92
6        2515         5889    0.81200        0.92
7        2515        15342    0.99350        0.92
8        2515        26899    0.99865        0.92
9        2515        26718    0.70035        0.92


In [137]:
new_data = data[['listing_id', 'reviewer_id', 'sentiment', 'true_score']]
print(new_data.shape)
new_data.drop(new_data[new_data.true_score == 0].index, inplace=True)
new_data.drop(new_data[new_data.sentiment == 0].index, inplace=True)
print(new_data.shape)
print(new_data.head(10))

(895016, 4)
(895016, 4)
   listing_id  reviewer_id  sentiment  true_score
0        2515         2603    0.96785        0.92
1        2515         8455    0.97685        0.92
2        2515         9759    0.98100        0.92
3        2515         9193    0.98130        0.92
4        2515         7048    0.96675        0.92
5        2515        12521    0.97505        0.92
6        2515         5889    0.81200        0.92
7        2515        15342    0.99350        0.92
8        2515        26899    0.99865        0.92
9        2515        26718    0.70035        0.92


In [138]:
new_data.to_csv('orig_data.csv')

In [152]:
data = pd.read_csv('orig_data.csv')
host_gender = pd.read_csv('output_gender_host_total.csv')
host_age = pd.read_csv('output_hosts_age_total.csv')

In [153]:
data = data.merge(host_gender, left_on='listing_id', right_on='ID')
data = data.filter(['listing_id','reviewer_id','sentiment', 'true_score','label'], axis=1)
data.rename(columns={'label':'host_gender'}, inplace=True)
# data = data.merge(host_age, left_on='listing_id', right_on='ID')
data = data.merge(host_age, left_on='listing_id', right_on='ID')
data = data.filter(['listing_id','reviewer_id', 'sentiment', 'true_score', 'host_gender', 'label'], axis=1)
data.rename(columns={'label':'host_age'}, inplace=True)
print(data.head(10))

   listing_id  reviewer_id  sentiment  true_score host_gender  host_age
0        2539     25160947    0.81245        0.97           F  (25, 32)
1        2539     91513326    0.96140        0.97           F  (25, 32)
2        2539     90022459    0.73770        0.97           F  (25, 32)
3        2539    116165195    0.96155        0.97           F  (25, 32)
4        2539    118432644    0.50000        0.97           F  (25, 32)
5        2539      1806142    0.99405        0.97           F  (25, 32)
6        2539    149989346    0.98190        0.97           F  (25, 32)
7        2595        50679    0.31940        0.97           M  (38, 43)
8        2595        53267    0.81245        0.97           M  (38, 43)
9        2595        38960    0.96240        0.97           M  (38, 43)


In [154]:
print(data.shape)
data.host_age = data.host_age.astype('str')
data = data[data.host_age != "(0, 2)"]
data = data[data.host_age != "(4, 6)"]
data = data[data.host_age != "(8, 12)"]
print(data.shape)

(128736, 6)
(121413, 6)


In [155]:
data.drop(data[data.host_gender == np.nan].index, inplace=True)
data.drop(data[data.host_age == np.nan].index, inplace=True)
print(data.head(10))
print(data.shape)

data.to_csv('host_focus_data.csv')

   listing_id  reviewer_id  sentiment  true_score host_gender  host_age
0        2539     25160947    0.81245        0.97           F  (25, 32)
1        2539     91513326    0.96140        0.97           F  (25, 32)
2        2539     90022459    0.73770        0.97           F  (25, 32)
3        2539    116165195    0.96155        0.97           F  (25, 32)
4        2539    118432644    0.50000        0.97           F  (25, 32)
5        2539      1806142    0.99405        0.97           F  (25, 32)
6        2539    149989346    0.98190        0.97           F  (25, 32)
7        2595        50679    0.31940        0.97           M  (38, 43)
8        2595        53267    0.81245        0.97           M  (38, 43)
9        2595        38960    0.96240        0.97           M  (38, 43)
(121413, 6)


In [156]:
data = pd.read_csv('orig_data.csv')
#Age
reviewer_age = pd.read_csv('output_age_reviewers_total.csv')
data = data.merge(reviewer_age, left_on='listing_id', right_on='ID')
data = data.filter(['listing_id','reviewer_id','sentiment', 'true_score', 'label'], axis=1)
data.rename(columns={'label':'reviewer_age'}, inplace=True)
data.reviewer_age = data.reviewer_age.astype('str')
data = data[data.reviewer_age != "(0, 2)"]
data = data[data.reviewer_age != "(4, 6)"]
data = data[data.reviewer_age != "(8, 12)"]

# # Gender
reviewer_gender = pd.read_csv('output_gender_reviewer_total.csv')
data = data.merge(reviewer_gender, left_on='listing_id', right_on='ID')
data = data.filter(['listing_id','reviewer_id','sentiment', 'true_score', 'reviewer_age','label'], axis=1)
data.rename(columns={'label':'reviewer_gender'}, inplace=True)

print(data.head(10))
print(data.shape)


# new_data = data[['listing_id', 'reviewer_id', 'sentiment', 'true_score']]
# print(new_data.shape)
# new_data.drop(new_data[new_data.true_score == 0].index, inplace=True)
# new_data.drop(new_data[new_data.sentiment == 0].index, inplace=True)
# print(new_data.shape)
# print(new_data.head(10))

   listing_id  reviewer_id  sentiment  true_score reviewer_age reviewer_gender
0      121473       592782    0.95160        0.95     (25, 32)               F
1      121473       660070    0.98720        0.95     (25, 32)               F
2      121473       660070    0.98630        0.95     (25, 32)               F
3      121473       498828    0.92775        0.95     (25, 32)               F
4      121473       749428    0.97010        0.95     (25, 32)               F
5      121473         9144    0.99550        0.95     (25, 32)               F
6      121473       161248    0.97195        0.95     (25, 32)               F
7      121473      1390425    0.95930        0.95     (25, 32)               F
8      121473       458760    0.98030        0.95     (25, 32)               F
9      121473      1787298    0.97880        0.95     (25, 32)               F
(542, 6)


In [157]:
data.to_csv('reviewer_focus_data.csv')